<a href="https://colab.research.google.com/github/SamuelaAnastasi/PrivateAiChallenge_ToyFederatedLearning/blob/master/PrivateAiChallenge_ToyFederatedLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Toy Federated Learning
Start by training a toy model the centralized way. We need:

* a toy dataset
* a model
* some basic training logic for training a model to fit the data.


In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 1.4MB 4.8MB/s 
     |████████████████████████████████| 276kB 43.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/16/27/a1/775c62ddea7bfa62324fd1f65847ed31c55dadb6051481ba3f
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 28053 (delta 6), reused 15 (delta 6), pack-reused 28034
Receiving objects: 100% (28053/28053), 31.84 MiB | 18.59 MiB/s, done.
Resolving deltas: 100% (18529/18529), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 5.0MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5

In [0]:
import torch as th
import syft as sy
from torch import nn, optim

In [0]:
hook = sy.TorchHook(th)

In [0]:
# Create local Dataset and target
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# Create a local linear Model
model = nn.Linear(2,1)

In [0]:
# define optimizer
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [7]:
# define training function and train model locally
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(1.5529)
tensor(0.8158)
tensor(0.5088)
tensor(0.3247)
tensor(0.2078)
tensor(0.1330)
tensor(0.0851)
tensor(0.0545)
tensor(0.0349)
tensor(0.0223)
tensor(0.0143)
tensor(0.0092)
tensor(0.0059)
tensor(0.0038)
tensor(0.0024)
tensor(0.0015)
tensor(0.0010)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)


In [0]:
# create 2 virtual workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
#send data and target to bob
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
#send data and target to alice
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
# define training function: model, optimizer 
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data 
            model = model.send(_data.location)

            # train remote model
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get model back after training
            model = model.get()

            print(loss.get())

In [14]:
# train model
train()

tensor(2.4100, requires_grad=True)
tensor(1.0599, requires_grad=True)
tensor(0.5514, requires_grad=True)
tensor(0.6654, requires_grad=True)
tensor(0.2942, requires_grad=True)
tensor(0.3891, requires_grad=True)
tensor(0.1691, requires_grad=True)
tensor(0.2266, requires_grad=True)
tensor(0.0977, requires_grad=True)
tensor(0.1320, requires_grad=True)
tensor(0.0565, requires_grad=True)
tensor(0.0770, requires_grad=True)
tensor(0.0327, requires_grad=True)
tensor(0.0449, requires_grad=True)
tensor(0.0189, requires_grad=True)
tensor(0.0263, requires_grad=True)
tensor(0.0109, requires_grad=True)
tensor(0.0154, requires_grad=True)
tensor(0.0063, requires_grad=True)
tensor(0.0090, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0053, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.0019, requires_grad=True)
tensor(0.0007, requires_grad=True)
tensor(0.0011, requires_grad=True)
tensor(0.0004, requi

#Lesson: Advanced Remote Execution Tools

In [37]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
# create tensor send to bob
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
#send pointer to alice
x = x.send(alice)

In [40]:
bob._objects

{3724733114: tensor([1, 2, 3, 4, 5])}

In [41]:
# x is a pointer to a pointer (alice)
x

(Wrapper)>[PointerTensor | me:44564907241 -> alice:21650725051]

In [42]:
# alice is a pointer to bob
alice._objects

{21650725051: (Wrapper)>[PointerTensor | alice:21650725051 -> bob:3724733114]}

In [0]:
y = x + x

In [45]:
y

(Wrapper)>[PointerTensor | me:39426306275 -> alice:14767770793]

In [25]:
alice._objects

{89116389133: (Wrapper)>[PointerTensor | alice:89116389133 -> bob:68368823764],
 89353432186: (Wrapper)>[PointerTensor | alice:89353432186 -> bob:97492511384]}

In [26]:
bob._objects

{68368823764: tensor([ 2,  4,  6,  8, 10]),
 97492511384: tensor([1, 2, 3, 4, 5])}

In [0]:
# clear data and pointer
bob.clear_objects()
alice.clear_objects()

# create tensor send to bob and send the pointer to alice using method chain
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [49]:
bob._objects

{70457532338: tensor([1, 2, 3, 4, 5])}

In [50]:
alice._objects

{32682272617: (Wrapper)>[PointerTensor | alice:32682272617 -> bob:70457532338]}

In [51]:
# get back pointer from alice now we point directly to bob
x = x.get()
x

(Wrapper)>[PointerTensor | me:32682272617 -> bob:70457532338]

In [52]:
bob._objects

{70457532338: tensor([1, 2, 3, 4, 5])}

In [53]:
alice._objects

{}

In [54]:
# get backe tensors from bob
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [55]:
# now bob got no more data
bob._objects

{}

In [0]:
# clear data and pointer
bob.clear_objects()
alice.clear_objects()

# create tensor send to bob and send the pointer to alice using method chain
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [57]:
bob._objects

{59378213065: tensor([1, 2, 3, 4, 5])}

In [58]:
alice._objects

{79945940891: (Wrapper)>[PointerTensor | alice:79945940891 -> bob:59378213065]}

In [0]:
# garbage collection default behaviour: deleting a pointer deletes related data and pointers
del x

In [60]:
bob._objects

{}

In [61]:
alice._objects

{}

#Lesson: Pointer Chain Operations

In [0]:
# clear data and pointer
bob.clear_objects()
alice.clear_objects()

# create tensor send to bob and send the pointer to alice using method chain
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [70]:
bob._objects

{48791182913: tensor([1, 2, 3, 4, 5])}

In [71]:
alice._objects

{48857320442: (Wrapper)>[PointerTensor | alice:48857320442 -> bob:48791182913]}

In [72]:
# command remote worker pointer (alice) to get data from bob
x.remote_get()

(Wrapper)>[PointerTensor | me:60579813913 -> alice:48857320442]

In [73]:
# now bob has no more data
bob._objects

{}

In [74]:
# because alice got them
alice._objects

{48857320442: tensor([1, 2, 3, 4, 5])}

In [75]:
# use convenience method to move data between remote workers: here from alice back to bob
x.move(bob)

(Wrapper)>[PointerTensor | me:60579813913 -> bob:60579813913]

In [76]:
bob._objects

{60579813913: tensor([1, 2, 3, 4, 5])}

In [80]:
alice._objects

{}